# Advanced Unix shell for genomics

This lesson will introduce some advanced concepts in the Unix shell. It will do so by guiding you through the creation of a simple RNA-seq data analysis pipeline.

You will be working with the Escherichia coli reference genome. You will remove the sequencing adapters, align some RNA-seq reads to it, and run some quality control checks.

You will begin by analysing the data one command at a time, and as new concepts are introduced you will make the pipeline increasingly automated.

Lesson formats:
- [git repository](https://github.com/bioinfo-lessons/adv-shell-genomics)
- [slides](https://bioinfo-lessons.github.io/adv-shell-genomics/)
- [HTML](https://bioinfo-lessons.github.io/adv-shell-genomics/notebook.html)

## Preparing the workspace

In any computational project, organisation of resources is essential.

In this lesson, the software you use will be managed with conda, leveraging the bioconda repository.

For storing data resources, including input data, output data, and log files, we will create a few directories to keep everything tidy

**NOTE**: it is highly recommended that you create a new conda environment for this lesson. The environment name that this demo uses is "adv-shell". You can choose another one if you like.

Create our working directory

In [8]:
mkdir -p ~/adv-shell
ls ~/

300.mzXML  Downloads         LICENSE           README.md            tmp
adv-shell  environment.yaml  miniconda3        samples_missing.csv  Zotero
bin        environment.yml   mozilla.pdf       software
Desktop    files             package-list.txt  src


Change into our working directory

In [1]:
cd ~/adv-shell
pwd

/home/tdido/adv-shell


Save our working directory in a variable for convenience

In [2]:
export WD=$(pwd)
echo $WD

/home/tdido/adv-shell


Create some base directories for our data

In [14]:
mkdir data log out res
ls -l

total 16
drwxr-xr-x 2 tdido tdido 4096 Oct 21 11:21 data
drwxr-xr-x 2 tdido tdido 4096 Oct 21 11:21 log
drwxr-xr-x 2 tdido tdido 4096 Oct 21 11:21 out
drwxr-xr-x 2 tdido tdido 4096 Oct 21 11:21 res


Let's see what we've done so far

In [13]:
pwd
tree

/home/tdido/adv-shell
.
├── data
├── log
├── out
└── res

4 directories, 0 files


## Obtaining our data

We will be working with real, publicly available sequencing data from an E. coli experiment.

In this experiment, the researchers over-expressed some DNA-damage handling proteins
and want to understand the effect of this change on RNA expression.

### Downloading the sequencing data

We will be analysing some E. coli RNA-seq data, which we will be downloading from the
[European Nucleotide Archive](https://www.ebi.ac.uk/ena/).

The ID of the experiment that we will use is *PRJEB29500*.

You should download both FASTQ (FTP) files for one of the samples. Make sure to place them in
your "data" directory.

In [70]:
cd $WD/data
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR286/002/ERR2868172/ERR2868172_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR286/002/ERR2868172/ERR2868172_2.fastq.gz

--2019-10-21 13:36:56--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR286/002/ERR2868172/ERR2868172_1.fastq.gz
           => ‘ERR2868172_1.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.192.7
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.192.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/ERR286/002/ERR2868172 ... done.
==> SIZE ERR2868172_1.fastq.gz ... 534742978
==> PASV ... done.    ==> RETR ERR2868172_1.fastq.gz ... done.
Length: 534742978 (510M) (unauthoritative)

ERR2868172_1.fastq. 100%[===================>] 509.97M  5.19MB/s    in 69s     

2019-10-21 13:38:11 (7.41 MB/s) - ‘ERR2868172_1.fastq.gz’ saved [534742978]

--2019-10-21 13:38:11--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR286/002/ERR2868172/ERR2868172_2.fastq.gz
           => ‘ERR2868172_2.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.192.7
Connecting to ftp.sra.ebi.ac.uk (ftp.s

Let's see what our work directory looks like

In [37]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   └── ERR2868172_2.fastq.gz
├── log
├── out
└── res

4 directories, 2 files


### Exploring the sequencing reads

We've donwloaded "paired-end" RNA-seq data. This means that every single RNA molecule is sequenced in two places, with a pre-defined gap in the middle. The two files we downloaded contain the first and the second read respectively, usually in matching order.
<img width="450" src="img/single_paired_end.png" />


In order to work with these files, we need to uncompress them. For this, we can use the "gunzip" command.

We would however like to keep the compressed files, so we also specify the "-k" (keep) argument. 

In [72]:
cd $WD/data
gunzip -k ERR2868172_1.fastq.gz
gunzip -k ERR2868172_2.fastq.gz
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq
│   ├── ERR2868172_1.fastq.gz
│   ├── ERR2868172_2.fastq
│   └── ERR2868172_2.fastq.gz
├── log
├── out
└── res

4 directories, 4 files


Let's look at the first read of each file to make sure they actually match.

**NOTE**: the FASTQ format uses 4 lines per read.

In [42]:
echo "read 1"
head -4 ERR2868172_1.fastq
echo "read 2"
head -4 ERR2868172_2.fastq

read 1
@ERR2868172.1 NS500589:161:H2CJ7BGX2:1:11101:26755:1045/1
GGCCGNAATTCCTACATCCTCGGTACTACATGNTTAGTCANNNNNTACATTCGCTTGCCAGCTGCGGACGGCCACG
+
A/AAA#AEAEEEA/EA/EAAEEA/AE/AE/<E#AE/EAE<#####EEE//E/EAEA<AA/EAAEEEE6AEE/A/A<
read 2
@ERR2868172.1 NS500589:161:H2CJ7BGX2:1:11101:26755:1045/2
CGGGGATCAAGAGAGGTCAAACCCAAAAGAGATCGCGTGGAAGCCCTGCCTGGGGTTGAAGCGTTACACCTTAATC
+
AAAAAAEEE/E/E/EE/E///EEEAA/<E/E<AEEEEEEEA/EEEEEEEEEEEEE<EE/6EEE/6E///E6AE/6A


Let's check the last read too, to be safe.

In [43]:
echo "read 1"
tail -4 ERR2868172_1.fastq
echo "read 2"
tail -4 ERR2868172_2.fastq

read 1
@ERR2868172.15966474 NS500589:161:H2CJ7BGX2:4:23612:17958:20394/1
GTCCGCAGGAATCCCGCGAATTTNCCAATCANCATCGNGNNTNGANGCTGTTTCCGAAATAAAATCAGGCAACGTT
+
AAAAAEEEEEEEEEEEEEEEEEE#EEEEEEE#EEEEE#E##E#EE#EEEEEEEEEEEEEEEEEEEEEEEEEEEEEA
read 2
@ERR2868172.15966474 NS500589:161:H2CJ7BGX2:4:23612:17958:20394/2
AGGGTGGTGGGGGCGGAGCGGGGGGAGTTGGGGGGGGGCGGGGGGCGGCTTTTTGGGCACAACACCAAAAAAGAAT
+
///////A//////A///6////A////A//////A/A//A</A//E/////E///////////E/A//EE/////


Let's see how many reads we have in our data.

To count the lines we can use the "wc" (word count) command, with the "-l" argument so that it only counts lines.

In [46]:
wc -l ERR2868172_1.fastq

63865896 ERR2868172_1.fastq


We know that the FASTQ format uses 4 lines per read, so we should divide the total lines in the
file by four.

In [47]:
expr 63865896 / 4

15966474


So apparently we have ~16 million reads in our files.

Even if we've got the answer we were looking for, there are a couple of problems with the approach we took:

- we had to manually copy the result of "wc -l" into a new command to get the read count.
- in order to look at the content of the files, we first uncompressed them. This is fine when dealing with a few files, but in a pipeline that may be processing hundreds of file this would be cumbersome to manage, time-consuming, and most importantly, storage-heavy.

Let's try to solve these issues with some shell tricks.

To solve the first problem, in order to count the number of reads from the file we could just keep one line per read and count that. The "grep" command allows us to filter the contents of a file by specifying a filtering statement.

The Unix shell has two "channels" of output: "standard output" (stdout) and "standard error" (stderr). Each piece of software can decide what to show on each channel at any time. This output will, by default, be shown on the screen. We can "redirect" stdout to a file by using the ">" operator.

We will then redirect the output of our grep command to a new file, which will contain only lines starting with "@" (the first line for each read).

In [56]:
grep "^@" ERR2868172_1.fastq > first_line
wc -l first_line

15966474 first_line


So there we have it: the number of reads without needing to manually enter data.

There is a hidden problem with that code, though. Even though the result is correct in this case, it turns out that line number 4, which contains the quality for each sequenced base, could also start with a "@". Therefore if we were unlucky and one of our reads had its first base with a quality represented as "@", we would be over-counting the reads.

To solve this, we can take advantage of the third line, which in FASTQ files from Illumina machines (like the ones we downloaded) is always just a "+" and nothing else.

In [57]:
grep "^+$" ERR2868172_1.fastq > first_line
wc -l first_line

15966474 first_line


To tackle the second problem, we would like to not have to uncompress our files. "gunzip" allows us to send the file contents to stdout instead of a file by specifying the "-c" argument.

Since showing the whole file on screen would be useless, we will combine "gunzip -c" with another command: "more". "more" is a command that allows you to view a file one screen at a time.

Since we are not going to uncompress the file, and therefore cannot use more against it directly, we need to somehow send the output of "gunzip -c" to "more". We can achieve this with what on Unix shell are known as "pipes".

"pipes" (which in the command line are represented simply by the character "|") allow you to send the standard output of one command to another command, without writing anything to a file. It is analogous to redirecting with ">" as we did before, but the destination is another command instead of a file.

Let's pipe the output of "gunzip -c" to more, in order to look at the file contents without uncompressing it first.

Since the second command in the pipe is receiving its input from the first one, it will not have an input file specified as an argument.

In [50]:
gunzip -c ERR2868172_1.fastq.gz | more

@ERR2868172.1 NS500589:161:H2CJ7BGX2:1:11101:26755:1045/1
GGCCGNAATTCCTACATCCTCGGTACTACATGNTTAGTCANNNNNTACATTCGCTTGCCAGCTGCGGACGGCCACG
+
A/AAA#AEAEEEA/EA/EAAEEA/AE/AE/<E#AE/EAE<#####EEE//E/EAEA<AA/EAAEEEE6AEE/A/A<
@ERR2868172.2 NS500589:161:H2CJ7BGX2:1:11101:15682:1046/1
GCTCANCCATGGCAACTAAAACGTACAGTACCNAGTAGTTNNANNCAGAGGCGAAACCGGCAAAACTGCCCCAGT
+
AAAAA#EEEEEEEEEEEEEEEEEEEEEEEEEE#EEEEEEE##E##EEEEEEEEEEEEAEEEEEEEEEEEEEEEEE
@ERR2868172.3 NS500589:161:H2CJ7BGX2:1:11101:25938:1048/1
GTCCGGACCCCGTTTTGAAAAAGACCAGGTCACCTGGCAGCAANTCGTCTTTATCAATTTGCGTGCCAATAGAGGC
+
A/AAAAEEEEEEAEEEEEEEEEEAEAE<AA/AEEEEEAEEEEE#EEEE<AAEEEEEEEEE/EEEAAEEEEEEEE/A
@ERR2868172.4 NS500589:161:H2CJ7BGX2:1:11101:23963:1049/1
CCTCTCTTGATCCCCGTCCTAAGAGCGGAGGCTAGGGAGAGAGGGCTCTAAGCAGGTTATTAAGCTGCTAAAGCGT
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@ERR2868172.5 NS500589:161:H2CJ7BGX2:1:11101:18187:1050/1
CACCGAACGAGAACATGATAATCGCCATCATCATCACCAGCCCGGTGAAGCCGTGCGGCAGGAAACCACCCTGATC
+
AAAAAEEEE

To further simplify our command, we can replace "gunzip -c" with the command "zcat" (zip + cat). "zcat" allows you to dump the content of a compresed file to stdout (similar to what "cat" does with uncompressed files).

In [51]:
zcat ERR2868172_1.fastq.gz | more

@ERR2868172.1 NS500589:161:H2CJ7BGX2:1:11101:26755:1045/1
GGCCGNAATTCCTACATCCTCGGTACTACATGNTTAGTCANNNNNTACATTCGCTTGCCAGCTGCGGACGGCCACG
+
A/AAA#AEAEEEA/EA/EAAEEA/AE/AE/<E#AE/EAE<#####EEE//E/EAEA<AA/EAAEEEE6AEE/A/A<
@ERR2868172.2 NS500589:161:H2CJ7BGX2:1:11101:15682:1046/1
GCTCANCCATGGCAACTAAAACGTACAGTACCNAGTAGTTNNANNCAGAGGCGAAACCGGCAAAACTGCCCCAGT
+
AAAAA#EEEEEEEEEEEEEEEEEEEEEEEEEE#EEEEEEE##E##EEEEEEEEEEEEAEEEEEEEEEEEEEEEEE
@ERR2868172.3 NS500589:161:H2CJ7BGX2:1:11101:25938:1048/1
GTCCGGACCCCGTTTTGAAAAAGACCAGGTCACCTGGCAGCAANTCGTCTTTATCAATTTGCGTGCCAATAGAGGC
+
A/AAAAEEEEEEAEEEEEEEEEEAEAE<AA/AEEEEEAEEEEE#EEEE<AAEEEEEEEEE/EEEAAEEEEEEEE/A
@ERR2868172.4 NS500589:161:H2CJ7BGX2:1:11101:23963:1049/1
CCTCTCTTGATCCCCGTCCTAAGAGCGGAGGCTAGGGAGAGAGGGCTCTAAGCAGGTTATTAAGCTGCTAAAGCGT
+
AAAAAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
@ERR2868172.5 NS500589:161:H2CJ7BGX2:1:11101:18187:1050/1
CACCGAACGAGAACATGATAATCGCCATCATCATCACCAGCCCGGTGAAGCCGTGCGGCAGGAAACCACCCTGATC
+
AAAAAEEEE

Now we can try to check the first read directly in compressed files.

In [52]:
zcat ERR2868172_1.fastq.gz | head -5

@ERR2868172.1 NS500589:161:H2CJ7BGX2:1:11101:26755:1045/1
GGCCGNAATTCCTACATCCTCGGTACTACATGNTTAGTCANNNNNTACATTCGCTTGCCAGCTGCGGACGGCCACG
+
A/AAA#AEAEEEA/EA/EAAEEA/AE/AE/<E#AE/EAE<#####EEE//E/EAEA<AA/EAAEEEE6AEE/A/A<
@ERR2868172.2 NS500589:161:H2CJ7BGX2:1:11101:15682:1046/1

gzip: stdout: Broken pipe


We can also do our counting without depending on extra files.

In [58]:
zcat ERR2868172_1.fastq.gz | grep "^+$" | wc -l

15966474


So it turns out that we didn't even need to uncompress our files after all.

Let's search our work directory to see where we have uncompressed fastq files.

In [60]:
find $WD -name "*.fastq"

/home/tdido/adv-shell/data/ERR2868172_2.fastq
/home/tdido/adv-shell/data/ERR2868172_1.fastq


Looks like it's just the two files that we uncompressed manually. It should therefore be safe enough to tell the "find" command to delete them.

We also don't need our helper file "first_line" anymore. Let's get rid of it as well.

In [63]:
find $WD -name "*.fastq" -delete
rm $WD/data/first_line

Let's check the state of our working directory, to make sure it's nice and clean.

In [64]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   └── ERR2868172_2.fastq.gz
├── log
├── out
└── res

4 directories, 2 files


### Doing some quality control checks on the data

In order to see if the data we are planning to use is good enough, we will use the "fastqc" software to do some quality control checks.

First, let's install fastqc.

In [89]:
conda install -y fastqc

Solving environment: done

## Package Plan ##

  environment location: /home/tdido/miniconda3/envs/adv-shell

  added / updated specs:
    - fastqc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.1.5             |    h516909a_1001         547 KB  conda-forge
    fastqc-0.11.8              |                1         9.6 MB  bioconda
    font-ttf-dejavu-sans-mono-2.37|       hab24e00_0         388 KB  conda-forge
    giflib-5.1.7               |       h516909a_1         455 KB  conda-forge
    lcms2-2.9                  |       h2e4bb80_0         423 KB  conda-forge
    libtiff-4.0.10             |    hfc65ed5_1004         574 KB  conda-forge
    lz4-c-1.8.3                |    he1b5a44_1001         187 KB  conda-forge
    openjdk-11.0.1             |    h46a85a0_1017       172.1 MB  conda-forge
    pthread-stubs-0.4          |    h14c3975_1001           5 KB  conda-forge
    

: 1

We can now run fastqc on our data.

In [91]:
cd $WD
mkdir out/fastqc
fastqc -o out/fastqc data/*.fastq.gz

(adv-shell) (adv-shell) Started analysis of ERR2868172_1.fastq.gz
Approx 5% complete for ERR2868172_1.fastq.gz
Approx 10% complete for ERR2868172_1.fastq.gz
Approx 15% complete for ERR2868172_1.fastq.gz
Approx 20% complete for ERR2868172_1.fastq.gz
Approx 25% complete for ERR2868172_1.fastq.gz
Approx 30% complete for ERR2868172_1.fastq.gz
Approx 35% complete for ERR2868172_1.fastq.gz
Approx 40% complete for ERR2868172_1.fastq.gz
Approx 45% complete for ERR2868172_1.fastq.gz
Approx 50% complete for ERR2868172_1.fastq.gz
Approx 55% complete for ERR2868172_1.fastq.gz
Approx 60% complete for ERR2868172_1.fastq.gz
Approx 65% complete for ERR2868172_1.fastq.gz
Approx 70% complete for ERR2868172_1.fastq.gz
Approx 75% complete for ERR2868172_1.fastq.gz
Approx 80% complete for ERR2868172_1.fastq.gz
Approx 85% complete for ERR2868172_1.fastq.gz
Approx 90% complete for ERR2868172_1.fastq.gz
Approx 95% complete for ERR2868172_1.fastq.gz
Analysis complete for ERR2868172_1.fastq.gz
Started analysis 

: 1

Let's see what happened.

In [92]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   └── ERR2868172_2.fastq.gz
├── log
├── out
│   └── fastqc
│       ├── ERR2868172_1_fastqc.html
│       ├── ERR2868172_1_fastqc.zip
│       ├── ERR2868172_2_fastqc.html
│       └── ERR2868172_2_fastqc.zip
└── res

5 directories, 6 files
(adv-shell) 

: 1

You can open the html files in your browser to view the reports.

### Making our data manageable in a workstation

As we've determined before, we are dealing with ~16 million sequencing reads. While your workstation would likely be able to deal with this amount of data, it would take longer than we can afford in the lesson.

We will therefore take a random sample of 300.000 reads, and use that for our downstream analysis.

To downsample our reads, we will use the "seqtk" program. "seqtk" is a toolkit for processing FASTQ and FASTA files.

We'll start by installing seqtk.

In [78]:
conda install -y seqtk

Solving environment: done

## Package Plan ##

  environment location: /home/tdido/miniconda3/envs/adv-shell

  added / updated specs:
    - seqtk


The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main
  libgcc-ng          pkgs/main/linux-64::libgcc-ng-9.1.0-hdf63c60_0
  seqtk              bioconda/linux-64::seqtk-1.3-hed695b0_2
  zlib               conda-forge/linux-64::zlib-1.2.11-h516909a_1006


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
(adv-shell) 

: 1

We would like to keep our original files safe, so let's put them in a subdirectory.

In [66]:
cd $WD/data
mkdir original
mv *.fastq.gz original
tree $WD

(adv-shell) (adv-shell) (adv-shell) /home/tdido/adv-shell
├── data
│   └── original
│       ├── ERR2868172_1.fastq.gz
│       └── ERR2868172_2.fastq.gz
├── log
├── out
│   └── fastqc
│       ├── ERR2868172_1_fastqc.html
│       ├── ERR2868172_1_fastqc.zip
│       ├── ERR2868172_2_fastqc.html
│       └── ERR2868172_2_fastqc.zip
└── res

6 directories, 6 files
(adv-shell) 

: 1

We can now subsample our files into the data directory.

In [85]:
seqtk sample -s100 original/ERR2868172_1.fastq.gz 300000 | gzip > ERR2868172_1.fastq.gz
seqtk sample -s100 original/ERR2868172_2.fastq.gz 300000 | gzip > ERR2868172_2.fastq.gz
ls -lah $WD/data
ls -lah $WD/data/original

(adv-shell) (adv-shell) total 25M
drwxr-xr-x 3 tdido tdido 4.0K Oct 21 14:34 .
drwxr-xr-x 6 tdido tdido 4.0K Oct 21 11:51 ..
-rw-r--r-- 1 tdido tdido  12M Oct 21 14:34 ERR2868172_1.fastq.gz
-rw-r--r-- 1 tdido tdido  13M Oct 21 14:35 ERR2868172_2.fastq.gz
drwxr-xr-x 2 tdido tdido 4.0K Oct 21 14:27 original
(adv-shell) total 1.1G
drwxr-xr-x 2 tdido tdido 4.0K Oct 21 14:27 .
drwxr-xr-x 3 tdido tdido 4.0K Oct 21 14:34 ..
-rw-r--r-- 1 tdido tdido 510M Oct 21 13:38 ERR2868172_1.fastq.gz
-rw-r--r-- 1 tdido tdido 540M Oct 21 13:39 ERR2868172_2.fastq.gz
(adv-shell) 

: 1

Let's also generate two new datasets that will be useful in later steps. We will do this just by varying the sampling seed, and assigning them fake names based on the real one.

**NOTE**: the repository where we downloaded the data contains samples named like the ones we're inventing. Don't get confused if you decide to download more samples from it.

In [96]:
seqtk sample -s7 original/ERR2868172_1.fastq.gz 300000 | gzip > ERR2868173_1.fastq.gz
seqtk sample -s7 original/ERR2868172_2.fastq.gz 300000 | gzip > ERR2868173_2.fastq.gz
seqtk sample -s56 original/ERR2868172_1.fastq.gz 300000 | gzip > ERR2868174_1.fastq.gz
seqtk sample -s56 original/ERR2868172_2.fastq.gz 300000 | gzip > ERR2868174_2.fastq.gz
ls -lah $WD/data
ls -lah $WD/data/original

(adv-shell) (adv-shell) (adv-shell) (adv-shell) total 73M
drwxr-xr-x 3 tdido tdido 4.0K Oct 21 14:45 .
drwxr-xr-x 6 tdido tdido 4.0K Oct 21 11:51 ..
-rw-r--r-- 1 tdido tdido  12M Oct 21 14:34 ERR2868172_1.fastq.gz
-rw-r--r-- 1 tdido tdido  13M Oct 21 14:35 ERR2868172_2.fastq.gz
-rw-r--r-- 1 tdido tdido  12M Oct 21 14:36 ERR2868173_1.fastq.gz
-rw-r--r-- 1 tdido tdido  13M Oct 21 14:36 ERR2868173_2.fastq.gz
-rw-r--r-- 1 tdido tdido  12M Oct 21 14:37 ERR2868174_1.fastq.gz
-rw-r--r-- 1 tdido tdido  13M Oct 21 14:37 ERR2868174_2.fastq.gz
drwxr-xr-x 2 tdido tdido 4.0K Oct 21 14:45 original
(adv-shell) total 1.1G
drwxr-xr-x 2 tdido tdido 4.0K Oct 21 14:45 .
drwxr-xr-x 3 tdido tdido 4.0K Oct 21 14:45 ..
-rw-r--r-- 1 tdido tdido 510M Oct 21 13:38 ERR2868172_1.fastq.gz
-rw-r--r-- 1 tdido tdido 540M Oct 21 13:39 ERR2868172_2.fastq.gz
(adv-shell) 

: 1

Let's check what our projects looks like with the newly generated data.

In [67]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   ├── ERR2868172_2.fastq.gz
│   ├── ERR2868173_1.fastq.gz
│   ├── ERR2868173_2.fastq.gz
│   ├── ERR2868174_1.fastq.gz
│   ├── ERR2868174_2.fastq.gz
│   └── original
│       ├── ERR2868172_1.fastq.gz
│       └── ERR2868172_2.fastq.gz
├── log
├── out
│   └── fastqc
│       ├── ERR2868172_1_fastqc.html
│       ├── ERR2868172_1_fastqc.zip
│       ├── ERR2868172_2_fastqc.html
│       └── ERR2868172_2_fastqc.zip
└── res

6 directories, 12 files
(adv-shell) 

: 1

## Downloading the genome

We will be downloading the Escherichia coli reference genome from the [NCBI](https://www.ncbi.nlm.nih.gov/).

We need to download the genome that matches the experiment that we will analyse. In order to identify the specific E. coli strain to download, we should check the metadata available on the ENA project where we obtained our data.

In [95]:
cd $WD
mkdir res/genome
cd $_
wget -O ecoli.fasta.gz ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
gunzip -k ecoli.fasta.gz

(adv-shell) (adv-shell) (adv-shell) --2019-10-21 14:47:21--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
           => ‘ecoli.fasta.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 2607:f220:41e:250::12
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2 ... done.
==> SIZE GCF_000005845.2_ASM584v2_genomic.fna.gz ... 1379902
==> PASV ... done.    ==> RETR GCF_000005845.2_ASM584v2_genomic.fna.gz ... done.
Length: 1379902 (1.3M) (unauthoritative)

GCF_000005845.2_ASM 100%[===================>]   1.32M  1.15MB/s    in 1.1s    

2019-10-21 14:47:24 (1.15 MB/s) - ‘ecoli.fasta.gz’ saved [1379902]

(adv-shell) (adv-shell) 

: 1

Let's see what our working directory looks like.

In [97]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   ├── ERR2868172_2.fastq.gz
│   ├── ERR2868173_1.fastq.gz
│   ├── ERR2868173_2.fastq.gz
│   ├── ERR2868174_1.fastq.gz
│   ├── ERR2868174_2.fastq.gz
│   └── original
│       ├── ERR2868172_1.fastq.gz
│       └── ERR2868172_2.fastq.gz
├── log
├── out
│   └── fastqc
│       ├── ERR2868172_1_fastqc.html
│       ├── ERR2868172_1_fastqc.zip
│       ├── ERR2868172_2_fastqc.html
│       └── ERR2868172_2_fastqc.zip
└── res
    └── genome
        ├── ecoli.fasta
        └── ecoli.fasta.gz

7 directories, 14 files
(adv-shell) 

: 1

And let's check what our genome looks like.

In [98]:
head -10 $WD/res/genome/ecoli.fasta

>NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTG
GTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGAC
AGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGT
AACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGG
TAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCG
ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTG
GCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTT
GACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAA
AACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAA
(adv-shell) 

: 1

## Analysing the data

We will now use the data we've prepared, performing the following steps:

- Remove sequencing adapters from our reads
- Indexing the genome for alignment
- Aligning our reads to the genome
- Generating a QC report

### Adapter trimming

During sequencing experiments, additional molecules are added to our molecules of interest in order for them to bind to our sequencing platform. These molecules are called *sequencing adapters*.

<img src="img/seq_synth.png" width="600">


Depending on the experiment, these sequencing adapters can actually be sequenced along with our molecules. We should then look for them and remove any we find.

To do this, we will use the "cutadapt" software.

Let's first install cutadapt.

In [101]:
conda install -y cutadapt

Solving environment: done

## Package Plan ##

  environment location: /home/tdido/miniconda3/envs/adv-shell

  added / updated specs:
    - cutadapt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    cutadapt-2.5               |   py37h516909a_0         171 KB  bioconda
    dnaio-0.4                  |   py37h516909a_0         130 KB  bioconda
    pip-19.3.1                 |           py37_0         1.9 MB  conda-forge
    setuptools-41.4.0          |           py37_0         634 KB  conda-forge
    wheel-0.33.6               |           py37_0          35 KB  conda-forge
    xopen-0.8.3                |           py37_0          15 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will b

: 1

We can now proceed to remove the adapters. We will also indicate that we want to discard any reads shorter than 20 nucleotides after trimming.

We will also be redirecting cutadapt's standard output to a log file.

**NOTE**: you can get the crazy long cutadapt command from [this link](https://gist.github.com/tdido).

In [106]:
cd $WD
mkdir out/cutadapt
mkdir log/cutadapt
cutadapt -m 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT \
-o out/cutadapt/ERR2868172_1.trimmed.fastq.gz \
-p out/cutadapt/ERR2868172_2.trimmed.fastq.gz \
data/ERR2868172_1.fastq.gz data/ERR2868172_2.fastq.gz > log/cutadapt/ERR2868172.log

[  8=--------] 00:00:06       300,000 reads  @     20.1 µs/read;   2.98 M reads/minute
(adv-shell) 

: 1

Let's check the logfile to see what happened.

In [ ]:
less log/cutadapt/ERR2868172.log

This is cutadapt 2.5 with Python 3.7.3
Command line parameters: -m 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o out/cutadapt/ERR2868172_1.trimmed.fastq.gz -p out/cutadapt/ERR2868172_2.trimmed.fastq.gz data/ERR2868172_1.fastq.gz data/ERR2868172_2.fastq.gz
Processing reads on 1 core in paired-end mode ...
Finished in 6.05 s (20 us/read; 2.97 M reads/minute).

=== Summary ===

Total read pairs processed:            300,000
  Read 1 with adapter:                   2,117 (0.7%)
  Read 2 with adapter:                   1,558 (0.5%)
Pairs that were too short:                 723 (0.2%)
Pairs written (passing filters):       299,277 (99.8%)

Total basepairs processed:    45,035,878 bp
  Read 1:    22,527,185 bp
  Read 2:    22,508,693 bp
Total written (filtered):     44,914,990 bp (99.7%)
  Read 1:    22,466,810 bp
  Read 2:    22,448,180 bp



### Indexing the genome and aligning our reads

In order to align our reads, we need to pre-process our genome by generating an index. This index will work just like an index in a book: it will allow us to find parts of the genome faster, without having to go through all of its bases.

Indexing is usually performed by the same software we use to align. In this case, we will be using STAR. Let's start by installing it.

In [4]:
conda install -y star

Solving environment: done

## Package Plan ##

  environment location: /home/tdido/miniconda3

  added / updated specs:
    - star


The following NEW packages will be INSTALLED:

  star               bioconda/linux-64::star-2.7.3a-0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
(base) 

: 1

We can now index our genome.

In [5]:
cd $WD
mkdir res/genome/star_index
STAR --runThreadN 4 --runMode genomeGenerate \
--genomeDir res/genome/star_index/ \
--genomeFastaFiles res/genome/ecoli.fasta \
--genomeSAindexNbases 9

(base) Oct 21 15:06:54 ..... started STAR run
Oct 21 15:06:54 ... starting to generate Genome files
Oct 21 15:06:54 ... starting to sort Suffix Array. This may take a long time...
Oct 21 15:06:54 ... sorting Suffix Array chunks and saving them to disk...
Oct 21 15:06:55 ... loading chunks from disk, packing SA...
Oct 21 15:06:55 ... finished generating suffix array
Oct 21 15:06:55 ... generating Suffix Array index
Oct 21 15:06:55 ... completed Suffix Array index
Oct 21 15:06:55 ... writing Genome to disk ...
Oct 21 15:06:55 ... writing Suffix Array to disk ...
Oct 21 15:06:55 ... writing SAindex to disk
Oct 21 15:06:55 ..... finished successfully
(base) 

: 1

With our index ready, we can now align our reads to it.

In [8]:
cd $WD
mkdir -p out/star/ERR2868172
STAR --runThreadN 4 --genomeDir res/genome/star_index/ \
--readFilesIn out/cutadapt/ERR2868172_1.trimmed.fastq.gz out/cutadapt/ERR2868172_2.trimmed.fastq.gz \
--readFilesCommand zcat \
--outFileNamePrefix out/star/ERR2868172/

(base) (base) Oct 21 15:09:05 ..... started STAR run
Oct 21 15:09:05 ..... loading genome
Oct 21 15:09:05 ..... started mapping
Oct 21 15:09:43 ..... finished mapping
Oct 21 15:09:43 ..... finished successfully
(base) 

: 1

Let's see what our project looks like.

In [9]:
tree $WD

/home/tdido/adv-shell
├── data
│   ├── ERR2868172_1.fastq.gz
│   ├── ERR2868172_2.fastq.gz
│   ├── ERR2868173_1.fastq.gz
│   ├── ERR2868173_2.fastq.gz
│   ├── ERR2868174_1.fastq.gz
│   ├── ERR2868174_2.fastq.gz
│   └── original
│       ├── ERR2868172_1.fastq.gz
│       └── ERR2868172_2.fastq.gz
├── log
│   └── cutadapt
│       └── ERR2868172.log
├── out
│   ├── cutadapt
│   │   ├── ERR2868172_1.trimmed.fastq.gz
│   │   └── ERR2868172_2.trimmed.fastq.gz
│   ├── fastqc
│   │   ├── ERR2868172_1_fastqc.html
│   │   ├── ERR2868172_1_fastqc.zip
│   │   ├── ERR2868172_2_fastqc.html
│   │   └── ERR2868172_2_fastqc.zip
│   └── star
│       └── ERR2868172
│           ├── Aligned.out.sam
│           ├── Log.final.out
│           ├── Log.out
│           ├── Log.progress.out
│           └── SJ.out.tab
└── res
    └── genome
        ├── ecoli.fasta
        ├── ecoli.fasta.gz
        └── star_index
            ├── chrLength.txt
            ├── chrNameLength.txt
            ├── chrName.txt
           

: 1

Let's check the STAR log file to see what it did.

In [10]:
more $WD/out/star/ERR2868172/Log.final.out

                                 Started job on |	Oct 21 15:09:05
                             Started mapping on |	Oct 21 15:09:05
                                    Finished on |	Oct 21 15:09:43
       Mapping speed, Million of reads per hour |	28.35

                          Number of input reads |	299277
                      Average input read length |	150
                                    UNIQUE READS:
                   Uniquely mapped reads number |	274739
                        Uniquely mapped reads % |	91.80%
                          Average mapped length |	149.72
                       Number of splices: Total |	189
            Number of splices: Annotated (sjdb) |	0
                       Number of splices: GT/AG |	93
                       Number of splices: GC/AG |	4
                       Number of splices: AT/AC |	0
               Number of splices: Non-canonical |	92
                      Mismatch rate per base, % |	0.13%
                         Deletion rate pe

Let's now look at the output alignments (the file is very wide, so you may be better off using a text editor).

In [3]:
head -10 $WD/out/star/ERR2868172/Aligned.out.sam

@HD	VN:1.4
@SQ	SN:NC_000913.3	LN:4641652
@PG	ID:STAR	PN:STAR	VN:2.7.3a	CL:STAR   --runThreadN 4   --genomeDir res/genome/star_index/   --readFilesIn out/cutadapt/ERR2868172_1.trimmed.fastq.gz   out/cutadapt/ERR2868172_2.trimmed.fastq.gz      --readFilesCommand zcat      --outFileNamePrefix out/star/ERR2868172/
@CO	user command line: STAR --runThreadN 4 --genomeDir res/genome/star_index/ --readFilesIn out/cutadapt/ERR2868172_1.trimmed.fastq.gz out/cutadapt/ERR2868172_2.trimmed.fastq.gz --readFilesCommand zcat --outFileNamePrefix out/star/ERR2868172/
ERR2868172.15617810	163	NC_000913.3	2755660	255	76M	=	2755694	110	TAAAAAGCCGCAAAAAATAGTCGCAAACGACGAAAACTACGCTTTAGCAGCTTAATAACCTGCTTAGAGCCCTCTC	AAAAAEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE	NH:i:1	HI:i:1	AS:i:150	nM:i:0
ERR2868172.15617810	83	NC_000913.3	2755694	255	76M	=	2755660	-110	AACTACGCTTTAGCAGCTTAATAACCTGCTTAGAGCCCTCTCTCCCTAGCCTCCGCTCTTAGGACGGGGATCAAGA	EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE

### Creating a report for the pipeline

In order to get an overview of what happened with the pipeline, and how well it worked, we'll create a report summarizing all the output. For this, we'll use the "MultiQC" software.

MultiQC parses a directory for output and log files from software it recognizes, and generates a report summarising its finds.

Let's start by installing it.

In [15]:
conda install -y multiqc

Solving environment: done

## Package Plan ##

  environment location: /home/tdido/miniconda3/envs/adv-shell

  added / updated specs:
    - multiqc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asn1crypto-1.2.0           |           py37_0         158 KB  conda-forge
    cffi-1.13.0                |   py37h8022711_0         220 KB  conda-forge
    libxkbcommon-0.9.1         |       hebb1f50_0         472 KB  conda-forge
    nspr-4.23                  |       he1b5a44_0         1.6 MB  conda-forge
    nss-3.47                   |       he751ad9_0         1.9 MB  conda-forge
    pyqt-5.12.3                |   py37hcca6a23_0         6.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.7 MB

The following NEW packages will be INSTALLED:

  asn1crypto         conda-forge/linux-64::asn1crypto-1.2.0

: 1

To run MultiQC on your workdir, we simply pass the workdir path to the "multiqc" binary.

In [16]:
multiqc -o out/multiqc $WD

/home/tdido/miniconda3/envs/adv-shell/lib/python3.7/site-packages/multiqc/utils/config.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  configs = yaml.load(f)
/home/tdido/miniconda3/envs/adv-shell/lib/python3.7/site-packages/multiqc/utils/config.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sp = yaml.load(f)
[INFO   ]         multiqc : This is MultiQC v1.7
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching '/home/tdido/adv-shell'
Searching 30 files..  [####################################]  100%          
[INFO   ]            star : Found 1 reports
[INFO   ]        cutadapt : Found 1 reports
[INFO   ]          fastqc : Found 2 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multi

: 1

## Building a pipeline

In this section we will be automating the steps that you just performed manually, so that they can be applied to any input files.

### Shell scripts

A shell script is a text file that contains shell commands, variables, and control structures.

Let's create a simple shell script that prints a name.

In [38]:
mkdir scripts
echo 'echo My name is Erwin Shroedinger' > scripts/print_name.sh

The script is hereby created. These are its contents:

In [34]:
cat scripts/print_name.sh

echo My name is Erwin Shroedinger


Let's run it then:

In [39]:
bash scripts/print_name.sh

My name is Erwin Shroedinger


Very exciting.

We can add another command to show the date when the script was run.

In [40]:
echo 'echo and this is the date: $(date)' >> scripts/print_name.sh
bash scripts/print_name.sh

My name is Erwin Shroedinger
and this is the date: Tue 22 Oct 12:57:32 CEST 2019


Notice how we redirect the second command using ">>" to append, instead of overwrite.

This is what the script contains right now:

In [41]:
cat scripts/print_name.sh

echo My name is Erwin Shroedinger
echo and this is the date: $(date)


### Exercise 1

Create a shell script that runs all the commands in the previous section, from cutadapt to the STAR alignment.

The name-printing script we wrote before works, but it's not very useful as it is.

Let's now modify it so that it takes the name as an argument.

In [22]:
echo 'echo My name is $1' > scripts/print_name_args.sh
bash scripts/print_name_args.sh "Max Planck"
#if we don't use the quotes, then we are passing two arguments and it will process only one
bash scripts/print_name_args.sh Max Planck

My name is Max Planck
My name is Max


We can make it more flexible by allowing it to take first and last name separately.

In [12]:
echo 'echo My name is $1 $2' > scripts/print_name_args_sep.sh
bash scripts/print_name_args_sep.sh Rosalind Franklin
bash scripts/print_name_args_sep.sh Marie Curie

My name is Rosalind Franklin
My name is Marie Curie


And we could also be more formal and write the last name first by switching the order of the variables.

In [13]:
echo 'echo My name is $2, $1' > scripts/print_name_args_sep_rev.sh
bash scripts/print_name_args_sep_rev.sh Rosalind Franklin
bash scripts/print_name_args_sep_rev.sh Marie Curie

My name is Franklin, Rosalind
My name is Curie, Marie


### Exercise 2

Modify your pipeline script so that it takes the sample ID as an argument.

The sample ID is part of the original filenames: filename = "ERR2868172_1.fastq.gz" => sample id = ERR2868172

### Control structures in the shell

Control structures are statements in programming languages that control the flow of the program depending on the outcome of certain conditions.

### The "if" control structure

"if" allows us to take one action or another depending on the result of a condition.

In [29]:
a=1
b=10

if [ "$a" -eq "$b" ]
then
    echo "$a is equal to $b"
else
    echo "$a is not equal to $b"
fi

1 is not equal to 10


In this case, we are comparing a and b for equality (-eq). Other possible numerical comparison operators are greater than (-gt), less than (-lt), and not equal to (-ne).

### Exercise 3

Modify your pipeline so that it prints usage information **and exits** if there is no input argument.

HINTS:

- Usage information messages are tipically formatted as "Usage: myscript.py myarg"
- You can force the script to exit with an error by adding the command "exit 1".
- You can test how many input arguments were passed by seeing if the special variable for the number of arguments, "$#", is equal to 1.

### The "for" control structure

"for" allows you to loop parts of your script for different values of a variable.

In [43]:
for name in {Erwin,Max,Rosalind,Marie}
do
    echo Hi, my name is $name
done

Hi, my name is Erwin
Hi, my name is Max
Hi, my name is Rosalind
Hi, my name is Marie


In [45]:
for i in {001..005}
do
    echo sample$i
done

sample001
sample002
sample003
sample004
sample005


### Control structures in the command line

Any shell command that lives in a script can also be used in the command line. The challenge is that you input things one line at a time, and long scripts can become complicated really quickly.

Some control structures are very useful though. As an example, let's see how we could use "for" to loop copy all our subsampled files to the "/tmp/datafiles" directory.

In [6]:
cd $WD
mkdir /tmp/datafiles
for file in data/*.fastq.gz; do cp $file /tmp/datafiles/; done;
ls /tmp/datafiles

ERR2868172_1.fastq.gz  ERR2868173_1.fastq.gz  ERR2868174_1.fastq.gz
ERR2868172_2.fastq.gz  ERR2868173_2.fastq.gz  ERR2868174_2.fastq.gz


### Exercise 4

Write a command line for loop that generates all sample ids from your data and executes your pipeline for each of them.

In [7]:
for sample in $(ls data/*.fastq.gz | cut -d"_" -f1 | sed "s:data/::" | sort | uniq); do echo $sample; done;

ERR2868172
ERR2868173
ERR2868174


### Exercise 5

Use the loop in the previous exercise to run your pipeline for all your samples.

### Exercise 6

Create a new script that uses a loop like the one in the previous exercise to run your pipeline script for all your samples.

See about moving the genome indexing code to this parent script, so that it is not run for every sample.

In addition, have the parent script run MultiQC after processing the samples, to generate the final report.